This uses a deep neural network and no imputation


In [36]:
!nvidia-smi

Mon Sep  4 15:18:47 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.69                 Driver Version: 384.69                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 20%   37C    P8    18W / 250W |  10691MiB / 11170MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
import numpy as np
from keras import backend as K
from keras.layers import Input, Embedding, merge
import keras.layers
from keras.regularizers import l2, l1
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.models import Sequential, Model
from keras.optimizers import SGD, RMSprop, Adam
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
#import pydotplus as pydot 
#import graphviz
from keras.utils import plot_model
from sklearn.metrics import mean_absolute_error

In [ ]:
import pandas as pd

In [ ]:
ratings_score_train = pd.read_csv('mal_scores_train_nonzero_v2.csv')# 
ratings_score_test = pd.read_csv('mal_scores_test_nonzero_v2.csv')# the v2 version of this data has no "test" users who aren't present in the training data. 
ratings_no_score_train = pd.read_csv('mal_scores_train_zero.csv') 

Now we need to re-order and dataprep.


In [ ]:
ratins_score_train

In [116]:
#userid mapping
users1 = ratings_score_train.userid.unique()
users2 = ratings_score_test.userid.unique()
users3 = ratings_no_score_train.userid.unique()
#animeid mapping
anime1= ratings_score_train.animeid.unique()
anime2=ratings_score_test.animeid.unique()
anime3=ratings_no_score_train.animeid.unique()
#statusid mapping
status = ratings_score_train.status.unique() #only need to do this on one of the dataframes because there are only 6 status, and a status cannot be missing. 
n_status = ratings_score_train.status.nunique()

In [117]:
users = set(users1).union(set(users2)).union(set(users3))

In [118]:
userid2idx=np.load("user.npy").item()
animeid2idx=np.load("anime.npy").item()
#userid2idx = {o:i for i,o in enumerate(users)} # turn the user strings into a numerical value. 

In [119]:
len(userid2idx)

230962

In [120]:
animes = set(anime1).union(set(anime2)).union(set(anime3))

In [121]:
#animeid2idx = {o:i for i,o in enumerate(animes)} # remove missing anime numbers and re-order

In [122]:
np.save("user.npy", userid2idx)
np.save("anime.npy", animeid2idx)


In [123]:
status2idx = {o:i for i,o in enumerate(status)} # turn the watched status into a vector 

In [124]:
n_users = len(users)
n_animes = len(animes)

In [125]:
ratings_score_test['anime_id_emb'] = ratings_score_test.animeid.apply(lambda x: animeid2idx[x])
ratings_score_train['anime_id_emb'] = ratings_score_train.animeid.apply(lambda x: animeid2idx[x])
ratings_no_score_train['anime_id_emb'] = ratings_no_score_train.animeid.apply(lambda x: animeid2idx[x])

In [126]:
ratings_score_test['user_id_emb'] = ratings_score_test.userid.apply(lambda x: userid2idx[x])
ratings_score_train['user_id_emb'] = ratings_score_train.userid.apply(lambda x: userid2idx[x])
ratings_no_score_train['user_id_emb'] = ratings_no_score_train.userid.apply(lambda x: userid2idx[x])

In [127]:
ratings_score_test['status_emb'] = ratings_score_test.status.apply(lambda x: status2idx[x])
ratings_score_train['status_emb'] = ratings_score_train.status.apply(lambda x: status2idx[x])
ratings_no_score_train['status_emb'] = ratings_no_score_train.status.apply(lambda x: status2idx[x])

In [128]:
n_factors = 40 #changing this number changes how many hidden factors each user and each anime is transformed into. 

In [129]:
ratings_score_train.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,animeid,animescore,index,name,status,unkey,userid,score,useravg,animeavg,score_anime_scaled,score_usr_scaled,user_rev_count,anime_id_emb,user_id_emb,status_emb
0,0,0,13202,2001,10,Lancerevomr,Lancerevomr,COMPLETED,2001Lancerevomr,Lancerevomr,10,10.0,8.681996,1.318004,0.0,1,1822,220189,0
1,1,1,21990,10719,10,kuelpanda,kuelpanda,COMPLETED,10719kuelpanda,kuelpanda,10,9.5,7.535472,2.464528,0.5,2,6344,53804,0
2,2,2,21991,10155,9,kuelpanda,kuelpanda,COMPLETED,10155kuelpanda,kuelpanda,9,9.5,7.105895,1.894105,-0.5,2,6092,53804,0
3,3,3,22409,15051,10,vxxyman,vxxyman,COMPLETED,15051vxxyman,vxxyman,10,10.0,7.654129,2.345871,0.0,1,7213,96692,0
4,4,4,23792,1,10,pinheiro,pinheiro,COMPLETED,1pinheiro,pinheiro,10,10.0,8.804023,1.195977,0.0,1,0,218194,0


In [130]:
ratings_score_train=ratings_score_train[(ratings_score_train['user_rev_count']<2000) & (ratings_score_train['user_rev_count']>10)]

In [131]:
ratings_score_train=ratings_score_train[ratings_score_train['score_usr_scaled']!=0]

In [109]:
ratings_score_train = ratings_score_train[ratings_score_train['anime_rev_count']<80000]# let's remove the absolutely most common. 

In [110]:
# these are the embedding functions that will be used for all models. For code modularity (the ability to cut paste a model to another notebook to test several at the same time), we're going to redfine this each time. 


In [142]:
def embedding_input_anime1(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_Anime_Hidden_Factors')(inp)
def embedding_input_user1(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_User_Hidden_Factors')(inp)

user_in1, u1 = embedding_input_user1('user_id_in', n_users+15, n_factors, 1e-7)
anime_in1, a1 = embedding_input_anime1('anime_id_in', n_animes, n_factors, 1e-8)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(229771, 32, input_length=1, name="Embed_User_Hidden_Factors", embeddings_regularizer=<keras.reg...)`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(12826, 32, input_length=1, name="Embed_Anime_Hidden_Factors", embeddings_regularizer=<keras.reg...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [143]:
# nn1, It is given no modifications for status. It is predicting score
x = merge([u1, a1], mode='concat', name='All_Factors_on_one_layer')
x = Flatten()(x)
x = Dropout(0.55, name='Prevent_overfit')(x)
x = Dense(70, activation='relu',name='Random_HF_Interactions')(x)
x = Dropout(0.55,name='Prevent_overfit2')(x)
x = Dense(16, activation='relu',name='Random_HF_Interactions2')(x)
x = Dense(1,name='Final_Interactions')(x)
nn1 = Model([user_in1, anime_in1], x)
nn1.compile(Adam(0.001), loss='mse')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [144]:
nn1.fit([ratings_score_train.user_id_emb, ratings_score_train.anime_id_emb], ratings_score_train.score, batch_size=5120, epochs=5, 
          validation_data=([ratings_score_test.user_id_emb, ratings_score_test.anime_id_emb], ratings_score_test.score))

Train on 9677648 samples, validate on 1210873 samples
Epoch 1/5
9677648/9677648 [==============================] - 23s - loss: 5.3648 - val_loss: 1.8508
Epoch 2/5
9677648/9677648 [==============================] - 22s - loss: 1.7931 - val_loss: 1.7974
Epoch 3/5
9677648/9677648 [==============================] - 22s - loss: 1.7159 - val_loss: 1.8321
Epoch 4/5
9677648/9677648 [==============================] - 22s - loss: 1.6857 - val_loss: 1.8526
Epoch 5/5
9677648/9677648 [==============================] - 22s - loss: 1.6598 - val_loss: 1.8505


In [135]:
nn1.fit([ratings_score_train.user_id_emb, ratings_score_train.anime_id_emb], ratings_score_train.score, batch_size=15120, epochs=8, 
          validation_data=([ratings_score_test.user_id_emb, ratings_score_test.anime_id_emb], ratings_score_test.score))

Train on 9677648 samples, validate on 1210873 samples
Epoch 1/8
9677648/9677648 [==============================] - 11s - loss: 1508.9978 - val_loss: 2149.6322
Epoch 2/8
9677648/9677648 [==============================] - 11s - loss: 2157.9858 - val_loss: 2149.4622
Epoch 3/8
9677648/9677648 [==============================] - 11s - loss: 2159.5891 - val_loss: 2149.3725
Epoch 4/8
9677648/9677648 [==============================] - 11s - loss: 2150.2961 - val_loss: 2149.0401
Epoch 5/8
9677648/9677648 [==============================] - 11s - loss: 2148.6452 - val_loss: 2148.7540
Epoch 6/8
9677648/9677648 [==============================] - 11s - loss: 2148.8633 - val_loss: 2148.4774
Epoch 7/8
9677648/9677648 [==============================] - 11s - loss: 2103.6858 - val_loss: 2089.7716
Epoch 8/8
9677648/9677648 [==============================] - 11s - loss: 2092.6544 - val_loss: 2089.5534


In [145]:
nn1pred= nn1.predict([ratings_score_test.user_id_emb, ratings_score_test.anime_id_emb])
nn1targ = ratings_score_test.score.values
mean_absolute_error(nn1targ, nn1pred)

1.01442604448999

In [146]:
nn1.save_weights('nn_score_weights_orig.h5')

In [ ]:
#plot_model(nn1, to_file='score.png')
#plot_model(nn12, to_file='user_score.png')
#plot_model(nn13, to_file='anime_score.png')

The NN2 Branch.
This drop all scores for shows that are not  listed as complete.


In [ ]:
ratings_score_train.head()

In [ ]:
ratings_score_train_complete= ratings_score_train[(ratings_score_train['status']=='COMPLETED') | (ratings_score_train['status']=='DROPPED')]

In [147]:
#ratings_score_train_complete= ratings_score_train # This Variation allows for hold, watching etc to also be used. 

In [149]:
def embedding_input_anime22(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_Anime_Hidden_Factors')(inp)
def embedding_input_user22(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_User_Hidden_Factors')(inp)

user_in22, u22 = embedding_input_user22('user_id_in', n_users+15, n_factors, 1e-7)
anime_in22, a22 = embedding_input_anime22('anime_id_in', n_animes, n_factors, 1e-8)
# nn22, It is only given complete. Trying to predict usr scaled score. 
x = merge([u22, a22], mode='concat', name='All_Factors_on_one_layer')
x = Flatten()(x)
x = Dropout(0.55, name='Prevent_overfit')(x)
x = Dense(70, activation='relu',name='Random_HF_Interactions')(x)
x = Dropout(0.55,name='Prevent_overfit2')(x)
x = Dense(16, activation='relu',name='Random_HF_Interactions2')(x)
x = Dense(1,name='Final_Interactions')(x)
nn22 = Model([user_in22, anime_in22], x)
nn22.compile(Adam(0.001), loss='mse')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(229771, 32, input_length=1, name="Embed_User_Hidden_Factors", embeddings_regularizer=<keras.reg...)`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(12826, 32, input_length=1, name="Embed_Anime_Hidden_Factors", embeddings_regularizer=<keras.reg...)`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use inst

In [150]:
nn22.fit([ratings_score_train_complete.user_id_emb, ratings_score_train_complete.anime_id_emb], ratings_score_train_complete.score_usr_scaled, batch_size=5120, epochs=5, 
          validation_data=([ratings_score_test.user_id_emb, ratings_score_test.anime_id_emb], ratings_score_test.score_usr_scaled))

Train on 9677648 samples, validate on 1210873 samples
Epoch 1/5
9677648/9677648 [==============================] - 24s - loss: 1.6767 - val_loss: 1.5965
Epoch 2/5
9677648/9677648 [==============================] - 23s - loss: 1.5972 - val_loss: 1.5794
Epoch 3/5
9677648/9677648 [==============================] - 23s - loss: 1.5766 - val_loss: 1.5782
Epoch 4/5
9677648/9677648 [==============================] - 23s - loss: 1.5657 - val_loss: 1.5838
Epoch 5/5
9677648/9677648 [==============================] - 23s - loss: 1.5580 - val_loss: 1.5905


In [151]:
nn22.fit([ratings_score_train_complete.user_id_emb, ratings_score_train_complete.anime_id_emb], ratings_score_train_complete.score_usr_scaled, batch_size=15120, epochs=8, 
          validation_data=([ratings_score_test.user_id_emb, ratings_score_test.anime_id_emb], ratings_score_test.score_usr_scaled))

Train on 9677648 samples, validate on 1210873 samples
Epoch 1/8
9677648/9677648 [==============================] - 11s - loss: 1.5429 - val_loss: 1.5795
Epoch 2/8
9677648/9677648 [==============================] - 11s - loss: 1.5386 - val_loss: 1.5797
Epoch 3/8
9677648/9677648 [==============================] - 11s - loss: 1.5361 - val_loss: 1.5803
Epoch 4/8
9677648/9677648 [==============================] - 11s - loss: 1.5335 - val_loss: 1.5766
Epoch 5/8
9677648/9677648 [==============================] - 11s - loss: 1.5310 - val_loss: 1.5695
Epoch 6/8
9677648/9677648 [==============================] - 11s - loss: 1.5282 - val_loss: 1.5710
Epoch 7/8
9677648/9677648 [==============================] - 11s - loss: 1.5258 - val_loss: 1.5658
Epoch 8/8
9677648/9677648 [==============================] - 11s - loss: 1.5232 - val_loss: 1.5680


In [152]:
nn22.save_weights('nn_score_usr_weights_orig.h5')

In [153]:
nn22pred= nn22.predict([ratings_score_test.user_id_emb, ratings_score_test.anime_id_emb])
nn22targ = ratings_score_test.score_usr_scaled.values
mean_absolute_error(nn22targ, nn22pred)

0.93747145176742608

In [154]:
def embedding_input_anime23(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_Anime_Hidden_Factors')(inp)
def embedding_input_user23(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_User_Hidden_Factors')(inp)

user_in23, u23 = embedding_input_user23('user_id_in', n_users+15, n_factors, 1e-7)
anime_in23, a23 = embedding_input_anime23('anime_id_in', n_animes, n_factors, 1e-8)
# nn23, It is only given complete. Trying to predict anime scaled score. 
x = merge([u23, a23], mode='concat', name='All_Factors_on_one_layer')
x = Flatten()(x)
x = Dropout(0.55, name='Prevent_overfit')(x)
x = Dense(70, activation='relu',name='Random_HF_Interactions')(x)
x = Dropout(0.55,name='Prevent_overfit2')(x)
x = Dense(16, activation='relu',name='Random_HF_Interactions2')(x)
x = Dense(1,name='Final_Interactions')(x)
nn23 = Model([user_in23, anime_in23], x)
nn23.compile(Adam(0.001), loss='mse')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(229771, 32, input_length=1, name="Embed_User_Hidden_Factors", embeddings_regularizer=<keras.reg...)`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(12826, 32, input_length=1, name="Embed_Anime_Hidden_Factors", embeddings_regularizer=<keras.reg...)`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use inst

In [155]:
nn23.fit([ratings_score_train_complete.user_id_emb, ratings_score_train_complete.anime_id_emb], ratings_score_train_complete.score_anime_scaled, batch_size=5120, epochs=5, 
          validation_data=([ratings_score_test.user_id_emb, ratings_score_test.anime_id_emb], ratings_score_test.score_anime_scaled))

Train on 9677648 samples, validate on 1210873 samples
Epoch 1/5
9677648/9677648 [==============================] - 23s - loss: 1.7921 - val_loss: 1.7744
Epoch 2/5
9677648/9677648 [==============================] - 22s - loss: 1.6563 - val_loss: 1.7464
Epoch 3/5
9677648/9677648 [==============================] - 22s - loss: 1.6281 - val_loss: 1.7353
Epoch 4/5
9677648/9677648 [==============================] - 22s - loss: 1.6107 - val_loss: 1.7507
Epoch 5/5
9677648/9677648 [==============================] - 22s - loss: 1.5987 - val_loss: 1.7530


In [156]:
nn23.fit([ratings_score_train_complete.user_id_emb, ratings_score_train_complete.anime_id_emb], ratings_score_train_complete.score_anime_scaled, batch_size=15120, epochs=8, 
          validation_data=([ratings_score_test.user_id_emb, ratings_score_test.anime_id_emb], ratings_score_test.score_anime_scaled))

Train on 9677648 samples, validate on 1210873 samples
Epoch 1/8
9677648/9677648 [==============================] - 11s - loss: 1.6468 - val_loss: 2.1152
Epoch 2/8
9677648/9677648 [==============================] - 11s - loss: 1.7907 - val_loss: 2.0473
Epoch 3/8
9677648/9677648 [==============================] - 11s - loss: 1.8625 - val_loss: 2.1534
Epoch 4/8
9677648/9677648 [==============================] - 11s - loss: 1.9112 - val_loss: 2.1735
Epoch 5/8
9677648/9677648 [==============================] - 11s - loss: 1.9717 - val_loss: 2.2579
Epoch 6/8
9677648/9677648 [==============================] - 11s - loss: 2.0448 - val_loss: 2.4646
Epoch 7/8
9677648/9677648 [==============================] - 11s - loss: 2.1702 - val_loss: 2.4314
Epoch 8/8
9677648/9677648 [==============================] - 11s - loss: 2.1903 - val_loss: 2.4573


In [157]:
nn23.save_weights('nn_score_anime_weights_orig.h5')

In [158]:
nn23pred= nn23.predict([ratings_score_test.user_id_emb, ratings_score_test.anime_id_emb])
nn23targ = ratings_score_test.score_anime_scaled.values
mean_absolute_error(nn23targ, nn23pred)

0.99841925557038202